## Run Gemini with structured output to get baseline results

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from hellofresh_extractor.llm.StructuredGeminiCaller import StructuredGeminiCaller
from hellofresh_extractor.llm.utils import convert_structured_result_to_df
from hellofresh_extractor.llm.prompts import multimodal_system_prompt, multimodal_user_query
from hellofresh_extractor.llm.output_schemas import ExtractedMeal
from hellofresh_extractor.gsuite.drive.GoogleDriveHelper import GoogleDriveHelper
import glob
from PIL import Image
from pillow_heif import register_heif_opener
from dotenv import load_dotenv
import time
import pandas as pd

In [ ]:
master_folder_name = "Recipes"
drive_helper = GoogleDriveHelper(f"{master_folder_name}")

In [ ]:
top_level_drive_folder_id = drive_helper.get_folder_id()

In [ ]:
load_dotenv()

In [ ]:
this_path = os.getcwd()
images_path = os.path.join(this_path,"images")

In [ ]:
images = glob.glob(os.path.join(images_path,"*.HEIC"))

In [ ]:
gemini_caller = StructuredGeminiCaller(
    api_key=os.environ.get("GEMINI_API_KEY"),
    model="gemini-2.5-flash-preview-04-17"
)

In [ ]:
len(images)

In [ ]:
all_meals = []
register_heif_opener()
t0 = time.time()
for i, image in enumerate(images):
    print("*"*30)
    print(f"At image {i}")
    open_image = Image.open(image).convert("RGB")
    result = gemini_caller.invoke(
        system_message=multimodal_system_prompt,
        input_content=[open_image, multimodal_user_query],
        output_schema=ExtractedMeal
    )

    if "structured_data" in result:
        analysis = result["structured_data"]
        df = convert_structured_result_to_df(analysis)
        df["image_path"] = image
        all_meals.append(df)
    else:
        print(f"Structured data field not found for result {result}")
t1 = time.time()
mean_process_time = (t1 - t0)/len(images)

In [ ]:
mean_process_time

In [ ]:
all_meals = pd.concat(all_meals)

In [ ]:
all_meals.to_csv("test_hello_fresh_recipes_gemini.csv",index=False)

In [ ]:
drive_helper.upload_csv_file(file_path="test_hello_fresh_recipes_gemini.csv",parent_folder_id=top_level_drive_folder_id)